# Julia project

In [1]:
using NPZ
using Gen
using Plots
using LinearAlgebra

In [2]:
xs = npzread("Z:/Master I/PP    - Probabilistic Programming/Project/input/input_10x10.npy")

10×10 Matrix{Float64}:
  8.0  6.0   8.0  0.0   4.0   6.0   6.0   6.0   0.0  6.0
  0.0  8.0  10.0  6.0  10.0   8.0   8.0   6.0   0.0  6.0
  0.0  6.0  10.0  0.0   6.0   6.0   8.0   6.0   0.0  8.0
  5.0  5.0   4.0  0.0   4.0   8.0   4.0   0.0   0.0  9.0
  8.0  6.0   6.0  0.0   6.0   6.0   6.0   0.0   0.0  6.0
  6.0  5.0   3.0  0.0   0.0   9.0   5.0   0.0   3.0  7.0
 10.0  0.0   6.0  0.0   6.0   6.0   6.0   0.0   6.0  0.0
 10.0  8.0   0.0  0.0   6.0   8.0   0.0   0.0   6.0  6.0
 10.0  0.0  10.0  0.0  10.0   0.0  10.0  10.0   0.0  8.0
  8.0  0.0   8.0  0.0   8.0  10.0   6.0   0.0  10.0  0.0

## Hierarchical Poisson Factorization

In [3]:
@gen function hpf_model(num_users::Int64,
                        num_items::Int64,
                        k::Int64,
                        a::Float64, c::Float64,
                        a_prim::Float64, b_prim::Float64, c_prim::Float64, d_prim::Float64)
    
    # for each user u
    X_preference = Vector[] # asta i theta care trebuie resimulat practic 
    for u = 1:num_users
        activity = ({(:activity,u)} ~ gamma(a_prim,a_prim/b_prim))
        preference = Float64[]
        for _k = 1:k  # for each component k ( k is like a random number of categories)
            push!(preference,{(:preference,u, _k)} ~ gamma(a,activity))
        end
        push!(X_preference,preference)
    end

    # for each item i 
    X_sample_attribute = Vector[] # asta i beta care trebuie resimulat
    for i = 1:num_items
        popularity = ({(:popularity,i)} ~ gamma(c_prim,c_prim/d_prim))
        attribute = Float64[]
        for _k = 1:k # for each component k ( k is like a random number of categories)
            push!(attribute,{(:attribute, i, _k)} ~ gamma(c,popularity))
        end
        push!(X_sample_attribute, attribute)
    end
    
    # for each user u and item i
    y = Matrix{Float64}(undef,0,num_items)
    for u = 1:num_users
        rating = Float64[]
        for i = 1:num_items
            push!(rating,{(:rating, u, i)} ~ poisson(dot(X_preference[u],X_sample_attribute[i])))
        end
        y = vcat(y, rating')
    end
    
    y
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Int64, Int64, Int64, Float64, Float64, Float64, Float64, Float64, Float64], false, Union{Nothing, Some{Any}}[nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing], var"##hpf_model#313", Bool[0, 0, 0, 0, 0, 0, 0, 0, 0], false)

### Test to see if it generates a matrix with ratings

In [4]:
num_users = 10
num_items = 10
k = 10 # pentru primele 10 filme sunt 10 categorii
a, c, a_prim, b_prim, c_prim, d_prim = [0.3 for _ = 1:6];
y = hpf_model(num_users, num_items, k, a, c, a_prim, b_prim, c_prim, d_prim );
y

10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  23.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0

In [5]:
trace = Gen.simulate(hpf_model, (num_users, num_items, k, a, c, a_prim, b_prim, c_prim, d_prim,))

Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}(DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Int64, Int64, Int64, Float64, Float64, Float64, Float64, Float64, Float64], false, Union{Nothing, Some{Any}}[nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing], var"##hpf_model#313", Bool[0, 0, 0, 0, 0, 0, 0, 0, 0], false), Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}((:rating, 6, 10) => Gen.ChoiceOrCallRecord{Int64}(0, -0.0002339270194919225, NaN, true), (:preference, 9, 9) => Gen.ChoiceOrCallRecord{Float64}(0.05758278885700929, 1.0115939047703972, NaN, true), (:rating, 9, 5) => Gen.ChoiceOrCallRecord{Int64}(0, -0.00014007203600753518, NaN, true), (:attribute, 4, 3) => Gen.ChoiceOrCallRecord{Float64}(0.001294617894415501, 4.725567062283881, NaN, true), (:rating, 6, 4) => Gen.ChoiceOrCallRecord{Int64}(0, -0.0006105420200919684, NaN, true), (:attribute, 4, 10) => Gen.ChoiceOrCallRecord{Float64}(0.013716732634152951, 2.27

### See if trace is corectly created

In [44]:
Gen.get_choices(trace)


│
├── (:rating, 6, 10) : 0
│
├── (:preference, 9, 9) : 0.4866740340294369
│
├── (:rating, 9, 5) : 0
│
├── (:attribute, 4, 3) : 0.3656949394884153
│
├── (:rating, 6, 4) : 0
│
├── (:attribute, 4, 10) : 0.01029092583548561
│
├── (:rating, 3, 10) : 0
│
├── (:rating, 4, 6) : 0
│
├── (:attribute, 8, 2) : 0.012665188145778942
│
├── (:activity, 6) : 0.07390284258122365
│
├── (:preference, 5, 5) : 0.012104156632052549
│
├── (:rating, 7, 1) : 0
│
├── (:rating, 3, 4) : 0
│
├── (:activity, 4) : 1.4104443881057465
│
├── (:rating, 7, 8) : 0
│
├── (:activity, 7) : 0.014351758031160936
│
├── (:preference, 8, 6) : 0.0005545263503797618
│
├── (:rating, 10, 7) : 0
│
├── (:rating, 2, 2) : 0
│
├── (:attribute, 7, 5) : 0.00024673750171484894
│
├── (:popularity, 10) : 0.1750755341133708
│
├── (:rating, 7, 2) : 0
│
├── (:attribute, 4, 4) : 0.11291890066067706
│
├── (:preference, 5, 6) : 6.495897080477486e-5
│
├── (:rating, 3, 9) : 0
│
├── (:preference, 1, 6) : 0.0014740892635608867
│
├── (:attribute, 3, 10) :

### Create constraints

In [45]:
function make_constraints(ys::Matrix{Float64})
    constraints = Gen.choicemap()
    for u=1:size(ys)[1]
        for i=1:size(ys)[2]  
        constraints[(:rating, u, i)] = ys[u,i]
        end
    end
    constraints
end;

Check if it works

In [46]:
make_constraints(xs)

│
├── (:rating, 6, 10) : 7.0
│
├── (:rating, 1, 1) : 8.0
│
├── (:rating, 9, 5) : 10.0
│
├── (:rating, 6, 4) : 0.0
│
├── (:rating, 8, 10) : 6.0
│
├── (:rating, 3, 10) : 8.0
│
├── (:rating, 4, 5) : 4.0
│
├── (:rating, 1, 9) : 0.0
│
├── (:rating, 4, 6) : 8.0
│
├── (:rating, 9, 4) : 0.0
│
├── (:rating, 1, 3) : 8.0
│
├── (:rating, 5, 10) : 6.0
│
├── (:rating, 3, 2) : 6.0
│
├── (:rating, 1, 10) : 6.0
│
├── (:rating, 5, 2) : 6.0
│
├── (:rating, 2, 6) : 8.0
│
├── (:rating, 6, 3) : 3.0
│
├── (:rating, 7, 1) : 10.0
│
├── (:rating, 3, 4) : 0.0
│
├── (:rating, 9, 8) : 10.0
│
├── (:rating, 7, 8) : 0.0
│
├── (:rating, 10, 10) : 0.0
│
├── (:rating, 5, 7) : 6.0
│
├── (:rating, 10, 5) : 8.0
│
├── (:rating, 5, 4) : 0.0
│
├── (:rating, 4, 8) : 0.0
│
├── (:rating, 10, 7) : 6.0
│
├── (:rating, 2, 2) : 8.0
│
├── (:rating, 2, 4) : 6.0
│
├── (:rating, 8, 5) : 6.0
│
├── (:rating, 8, 1) : 10.0
│
├── (:rating, 7, 2) : 0.0
│
├── (:rating, 2, 10) : 6.0
│
├── (:rating, 3, 9) : 0.0
│
├── (:rating, 10, 9) : 10.0
│
├─

In [47]:
function block_resimulation_update(tr,num_users, num_items, k)

    # Block 1: Update preference (theta)
    for u = 1:num_users
        for _k = 1:k 
        latent_variable = select(:preference,u,_k)
        (tr, _) = mh(tr, latent_variable)
        end
    end


    # Block 2: Update attribute (beta)
    for i = 1:num_items
        for _k = 1:k
        latent_variable = select(:attribute,i,_k)
        (tr, _) = mh(tr, latent_variable)
        end
    end

    tr

end

block_resimulation_update (generic function with 1 method)

In [48]:
function block_resimulation_inference(n_burnin, n_samples, thin)
    observations = make_constraints(xs)
    (tr, _) = generate(hpf_model, (num_users, num_items, k, a, c, a_prim, b_prim, c_prim, d_prim), observations)
    for iter=1:n_burnin
        tr = block_resimulation_update(tr,num_users,num_items,k)
        if iter % 100 == 0 
            print(iter)
        end

    end
    trs = []
    for iter=1:n_samples
        for itert = 1:thin # reduce the autocorrelation in a sample of generated data by MCMC
            tr = block_resimulation_update(tr,num_users,num_items,k)
        end
        push!(trs, tr)
        if iter % 100 == 0 
            print(iter)
        end
    end

    trs

end;

### TRAIN

In [49]:
n_burnin = 100000
n_samples = 40000
thin = 2
trs = block_resimulation_inference(n_burnin, n_samples, thin);

100

200300

400

500600

700800

...

3940039500

3960039700

39800

39900

40000

In [50]:
function create_rating_vector(one_trace,num_users,num_items,k)
    X_preference = Vector[] 
    for u = 1:num_users
        preference = Float64[]
        for _k = 1:k  
            push!(preference,one_trace[(:preference,u,_k)])
        end

        push!(X_preference,preference)
    end

    X_sample_attribute = Vector[] 
    for i = 1:num_items
        attribute = Float64[]
        for _k = 1:k 
            push!(attribute,one_trace[(:attribute, i, _k)])
        end
        push!(X_sample_attribute, attribute)
    end
    y = Matrix{Float64}(undef,0,num_items)
    for u = 1:num_users
        rating = Float64[]
        for i = 1:num_items
            push!(rating,poisson(dot(X_preference[u],X_sample_attribute[i])))
        end
        y = vcat(y, rating')
    end
     
    y

end



create_rating_vector (generic function with 1 method)

Compute matrix of rating for each trace 

In [51]:
final_matrix = zeros(num_users, num_items)
for i=1:n_samples
    y = create_preference_vector(trs[i],num_users,num_items,k)
    final_matrix = final_matrix + y
end
final_matrix/n_samples


10×10 Matrix{Float64}:
 5.0e-5  0.03755   0.286125  0.04735   …  0.0236    0.0148    0.162075
 2.5e-5  0.10555   0.085225  0.1516       0.647425  0.038325  0.602075
 0.0     0.010475  0.03625   0.025075     0.032775  0.005125  0.02295
 0.0     0.00015   0.000875  0.005675     0.000675  5.0e-5    0.0012
 0.0     0.000175  0.000675  0.0001       5.0e-5    0.000125  0.003275
 0.0     0.11015   0.358525  0.028825  …  0.264375  0.0856    1.37005
 0.0     0.0       0.0       0.0          0.0       0.0       0.0
 0.0     7.5e-5    0.00015   7.5e-5       2.5e-5    0.0       5.0e-5
 0.0     0.000775  0.0099    0.007775     0.0039    0.00085   0.00745
 0.0     0.0       5.0e-5    2.5e-5       2.5e-5    0.0       0.0

In [52]:
xs

10×10 Matrix{Float64}:
  8.0  6.0   8.0  0.0   4.0   6.0   6.0   6.0   0.0  6.0
  0.0  8.0  10.0  6.0  10.0   8.0   8.0   6.0   0.0  6.0
  0.0  6.0  10.0  0.0   6.0   6.0   8.0   6.0   0.0  8.0
  5.0  5.0   4.0  0.0   4.0   8.0   4.0   0.0   0.0  9.0
  8.0  6.0   6.0  0.0   6.0   6.0   6.0   0.0   0.0  6.0
  6.0  5.0   3.0  0.0   0.0   9.0   5.0   0.0   3.0  7.0
 10.0  0.0   6.0  0.0   6.0   6.0   6.0   0.0   6.0  0.0
 10.0  8.0   0.0  0.0   6.0   8.0   0.0   0.0   6.0  6.0
 10.0  0.0  10.0  0.0  10.0   0.0  10.0  10.0   0.0  8.0
  8.0  0.0   8.0  0.0   8.0  10.0   6.0   0.0  10.0  0.0